# 🔍 Agentic RAG with Azure AI Foundry (Python)

## 📋 Learning Objectives

This notebook demonstrates how to build intelligent Retrieval-Augmented Generation (RAG) systems using the Microsoft Agent Framework with Azure AI Foundry. You'll learn to create agents that can search through documents and provide accurate, context-aware responses.

**What You'll Build:**
- 📚 **Document-Aware Agents**: Agents that can search and reason over your documents
- 🔍 **Semantic Search**: Vector-based document retrieval with relevance ranking
- 🧠 **Context Integration**: Combining retrieved content with AI reasoning
- 💬 **Interactive Q&A**: Natural language querying over document collections

## 🎯 RAG Architecture Concepts

### Core RAG Components
- **Document Ingestion**: Upload and process documents for search
- **Vector Store**: Semantic embedding storage and retrieval system
- **Search Tool**: Intelligent document chunk retrieval
- **Agent Integration**: Seamless combination of search and generation

### Azure AI Foundry Benefits
- **Managed Infrastructure**: Enterprise-grade vector store management
- **Automatic Indexing**: Real-time document processing and embedding
- **Scalable Search**: High-performance semantic search capabilities
- **Integrated Security**: Built-in authentication and access controls

## 🔧 Technical Implementation

### RAG Processing Pipeline
```python
Document Upload → Vector Embedding → Index Creation → Search Query
                                    ↓
User Question → Semantic Search → Retrieved Context → AI Response
```

### Key Components
- **Azure AI Foundry**: Managed RAG infrastructure and services
- **Vector Store**: Semantic search and document storage
- **File Search Tool**: Intelligent document chunk retrieval
- **Agent Framework**: Orchestration of search and generation

## 📊 Use Cases & Applications

### 1. **Knowledge Base Q&A**
- Internal company documentation search
- Product manual and FAQ systems
- Research paper analysis and discovery
- Technical documentation assistance

### 2. **Content Discovery**
- Academic research assistance
- Legal document analysis
- Medical literature review
- Business intelligence reporting

### 3. **Customer Support**
- Automated help desk responses
- Product support documentation
- Troubleshooting guide navigation
- Service manual consultation

## ⚙️ Prerequisites & Setup

**Azure Requirements:**
- Azure AI Foundry project workspace
- Azure CLI authentication configured
- Azure Developer CLI authentication configured
- Appropriate permissions for document upload and vector store creation

## ⚙️ Prerequisites & Setup

**Required Dependencies:**
```bash

pip uninstall agent-framework -y
pip uninstall agent-framework-azure-ai -y

pip install -r ../../Installation/requirements.txt --constraint ../../Installation/constraints.txt -U
```

**Authentication Setup:**
```bash
az login  # Authenticate with Azure CLI
az account set --subscription "your-subscription-id"
azd auth login
```

**Document Preparation:**
- Supported formats: Markdown, PDF, text files, Word documents
- Recommended file size: Under 100MB per document
- Content optimization: Clear headings and structured text

## 🏗️ RAG Architecture Patterns

### Document Processing
- **Chunking Strategy**: Intelligent text segmentation for optimal retrieval
- **Embedding Generation**: High-quality vector representations
- **Metadata Extraction**: Document structure and context preservation
- **Index Optimization**: Performance tuning for search operations

### Search & Retrieval
- **Semantic Matching**: Beyond keyword search with contextual understanding
- **Relevance Ranking**: Advanced scoring algorithms for result ordering
- **Context Windowing**: Optimal context size for generation quality
- **Multi-Document Search**: Cross-document information synthesis

### Integration Patterns
- **Real-Time Updates**: Dynamic document addition and removal
- **Security Controls**: Access management and content filtering
- **Performance Monitoring**: Search quality and response time tracking
- **Error Handling**: Graceful degradation and failure recovery

## 🚀 Advanced Features

- **Multi-Modal RAG**: Support for images, tables, and structured content
- **Conversational RAG**: Context-aware multi-turn document discussions
- **Federated Search**: Querying across multiple document collections
- **Real-Time Indexing**: Immediate availability of newly uploaded content

Ready to build intelligent document-powered agents? Let's create a smart knowledge assistant! 📖✨

In [ ]:
# 📦 Import Required Libraries for RAG Implementation

import os  # Environment variable access for configuration

# 🔍 Azure AI Agents Components for RAG Functionality
from azure.ai.agents.models import FilePurpose, VectorStore, FileSearchTool  # Core RAG models
from azure.identity.aio import AzureCliCredential  # Azure authentication for secure access
from dotenv import load_dotenv  # Environment variable management

# 🤖 Agent Framework Components for RAG Integration
from agent_framework import AgentRunResponse, ChatAgent, HostedFileSearchTool, HostedVectorStoreContent
from agent_framework.azure import AzureAIAgentClient  # Unified Azure AI client for RAG operations

In [ ]:
# 🔧 Load Environment Configuration for RAG Setup
# Initialize environment variables for Azure AI Foundry access
# Note: Azure AI Foundry uses Azure CLI credentials, so API keys are not required in .env
load_dotenv()

True

In [3]:
async def create_vector_store(client: AzureAIAgentClient) -> tuple[str, VectorStore]:
    """Create a vector store with sample documents."""
    file_path = './document.md'
    file = await client.project_client.agents.files.upload_and_poll(file_path=file_path, purpose=FilePurpose.AGENTS)
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.project_client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, vector_store

In [4]:
async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        file_id, vector_store = await create_vector_store(chat_client)

        file_search = FileSearchTool(vector_store_ids=[vector_store.id])
        
        agent = chat_client.create_agent(
            name="PythonRAGDemo",
            instructions="""
                You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                """,
            tools=file_search.definitions,  # Tools available to the agent
            tool_resources=file_search.resources,  # Resources for the tool
        )
                

        print("Agent created. You can now ask questions about the uploaded document.")

        query = "Can you explain Contoso's travel insurance coverage?"
        response = await AgentRunResponse.from_agent_response_generator(agent.run_stream(query, tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}))
        print(f"Assistant: {response}")

Uploaded file, file ID: assistant-PFvtKYcMvWaenGiA6dGmtd
Created vector store, ID: vs_RhKenp1evPBK9JLtj779aS4o
Agent created. You can now ask questions about the uploaded document.
Assistant: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage. This coverage is part of their premium travel services, which also feature luxury vacation packages, personalized itinerary planning, and 24/7 concierge support. The insurance is designed to support travelers on trips to exotic destinations worldwide, such as the Maldives, Swiss Alps, and African safaris【4:0†document.md】.
